Imports
==============================================================================================

In [ ]:
#%matplotlib inline
import numpy as np
import pandas as pd
from keras.callbacks import TensorBoard, History
from keras.layers import Input, Dense, Dropout
from keras.utils import plot_model
from keras.models import Model
from pathlib import Path
import sys
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
pd.options.mode.chained_assignment = None  # default='warn'
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import math
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from evaluation import *
from machineLearningModel import *

Configuration
==============================================================================================

In [ ]:
# fix random seed for reproducibility
np.random.seed(13)

## net params
num_layers = 3#6
num_neurons = 300#500
batch_size = 100#500#1000
dropout_rate = 0
#const_features = ['latitude', 'longitude', 'altitude', 'modules_per_string', 'strings_per_inverter', 'tilt',
#                  'azimuth', 'albedo', 'Technology', 'BIPV', 'A_c', 'N_s', 'pdc0', 'gamma_pdc', 'SystemID']#15
#dyn_features = ['Wind Direction_x', 'Wind Direction_y', 'Total Cloud Cover', 'Low Cloud Cover', 'Medium Cloud Cover',
#                'High Cloud Cover', 'Wind Speed', 'Wind Gust', 'Total Precipitation',
#                'Snow Fraction', 'Mean Sea Level Pressure', 'DIF - backwards', 'DNI - backwards', 'Shortwave Radiation',
#                'Temperature', 'Relative Humidity', 'Hour_x', 'Hour_y', 'Month_x', 'Month_y']#20
const_features = ['SystemID']
dyn_features = ['DIF - backwards', 'DNI - backwards', 'Shortwave Radiation', 'Temperature', 'Relative Humidity', 'Hour_x', 'Hour_y', 'Month_x', 'Month_y']
target_features = ['power']
drop_features = ['power_pvlib']
act_fct = 'relu'
out_act = 'linear'
loss_fct = 'mae'
optim = 'adam'
metrics = []
history = History()
val_history = History()

## data params
filename = './data/full_data_5_systems.csv'
correlations = ['pearson']#'pearson', 'spearman', 'kendall']
timesteps = 5#24
shape = (len(const_features) + len(dyn_features) + timesteps * (len(dyn_features) + len(target_features)),)# - 1

## training params
tensorboard = False
shuffle = True
epochs = 5#20
val_split = 1.0/10.0
forecast_horizon = 24
sliding_window = 24#168#24
dir = './test_results/'
if not os.path.exists(dir):
    os.makedirs(dir)

Data Preprocessing
==============================================================================================

In [ ]:
pfname = dir + 'preprocessed_data_t-'+str(timesteps)+'_f'+str(shape[0])+'.csv'
print(pfname)
prep = Path(pfname)
if prep.exists():
    print('Loading preprocessed dataset ...')
    pvlib = np.array_split(pd.read_csv(filename, skipinitialspace=True).set_index('time'), 5)[-1].power_pvlib
    dataset = pd.read_csv(pfname, skipinitialspace=True).set_index(['time', 'SystemID'])
else:
    print('Data preprocessing ...')
    df = pd.read_csv(filename, skipinitialspace=True).set_index('time')
    df = np.array_split(df, 5)[-1] ##################################
    pvlib = df.power_pvlib
    dataset = df[const_features + dyn_features + target_features].copy()[:'2017-02-09 10:00:00']

    #separate system
    for i in range(1, timesteps + 1):
        for feature in dyn_features + target_features:
            sys.stdout.write("Shifting %i/%i %s                \r" % (i, timesteps, feature))
            sys.stdout.flush()
            dataset[feature + ' t-' + str(i)] = dataset.shift(i)[feature]
    print('Shifting done.                ')

    dataset['forecast_horizon'] = 0
    p = dataset[target_features]
    dataset = dataset.drop(target_features, axis=1)
    for f in target_features:
        dataset[f] = p[f]
    dataset = dataset.dropna().reset_index().set_index(['time', 'SystemID'])

    sys.stdout.write("Writing to file ...\r")
    sys.stdout.flush()
    dataset.to_csv(pfname, encoding='utf-8')
    print('Writing done.                ')

    if correlations:
        sys.stdout.write('Computing correlations ...\r')
        sys.stdout.flush()
        for corr in correlations:
            sys.stdout.write("Computing %s correlation matrix                \r" % (corr))
            sys.stdout.flush()
            dataset.corr(method=corr).to_csv(dir + corr + '_correlations.csv', encoding='utf-8')
        print('Correlations done.                   ')

train, test = dataset[:('2015-10-12 06:00:00', 4.0)], dataset[('2015-10-12 07:00:00', 4.0):]
trainX, trainY = train.iloc[:,:-len(target_features)], train.iloc[:,-len(target_features):]
testX, testY = test.iloc[:,:-len(target_features)], test.iloc[:,-len(target_features):]
idx = testX.index.values

Build Model
==============================================================================================

In [ ]:
if tensorboard:
    print('tensorboard activated')
    callbacks = [TensorBoard(log_dir='./tensorboard', histogram_freq=1, batch_size=batch_size, write_graph=True, write_grads=True, write_images=False), history]
else:
    callbacks = [history]

#model = SARIMAX((0,1,2), (1,0,0,24))
model = MultiLayerPerceptron(shape, len(target_features), num_layers, num_neurons, loss_fct, optim,
                 act_fct, out_act, metrics, dropout_rate, dir + 'model.png', batch_size,
                 epochs, val_split, callbacks, 1, True)

Training
==============================================================================================

In [ ]:
X = trainX
y = trainY
if shuffle:
    df = pd.DataFrame(np.concatenate((trainX, trainY), axis=1))
    df = df.sample(frac=1).values
    y = df[:, -len(target_features):]
    X = df[:, :-len(target_features)]

model.learn(X, y)

name = './saved_models/pretrained_t-'+str(timesteps)+'_f'+str(shape[0])+'_e'+str(epochs)+'_b'+str(batch_size)+'_sys'+str(4)
# serialize model to JSON
model_json = model.model.to_json()
with open(name + ".json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.model.save_weights(name + ".h5")
print("Saved model to disk")

Walk-Forward Validation
==============================================================================================

In [ ]:
method = 'pvlib'

model.epochs = 5
model.validation_split = 0.0
model.batch_size = sliding_window
model.verbose = 0

#predictions = []
#for i in range(len(testX)):
#    sys.stdout.write("Walk-Forward Validation %i/%i\r" % (i+1, len(testX)))
#    sys.stdout.flush()
#    predictions.append(pd.DataFrame(model.forecast(testX.iloc[i:i+1,:])))
#    window = i - sliding_window
#    if window < 0:
#        window = 0
#    model.callbacks = [val_history]
#    model.learn(testX.iloc[window:i+1,:], testY.iloc[window:i+1,:])
#prediction = pd.concat(predictions)





forecast_horizon = 3
predictions = []
length = len(testX) - forecast_horizon - 1     #-11000
for i in range(length):# - 10
    sys.stdout.write("Walk-Forward Validation %i/%i\r" % (i+1, length))
    sys.stdout.flush()
    
    # initialize values for lagged power columns
    p = []
    for l in range(1, timesteps + 1):
        p.append(testX.iloc[i:i+1,:]['power t-'+str(l)].values[0])
        
    ps = []
    for f in range(forecast_horizon):
        # build input vector for future timestep
        t = testX.iloc[i+f:i+1+f,:].copy()
        for l in range(timesteps-1, 1, -1):
            t['power t-' + str(l+1)] = p[l]
            p[l] = p[l-1]
        t['power t-1'] = p[0]
        
        # make prediction for input new vector
        p[0] = model.forecast(t).item(0)
        ps.append(p[0])
    predictions.append(pd.DataFrame(ps))
    
    # train with newly available data
    window = i - sliding_window
    if window < 0:
        window = 0
    model.callbacks = [val_history]
    model.learn(testX.iloc[window:i+1,:], testY.iloc[window:i+1,:])
prediction = pd.concat(predictions)

#name = './saved_models/trained_t-'+str(timesteps)+'_f'+str(shape[0])+'_e'+str(epochs)+'_b'+str(batch_size)+'_sys'+str(4)
# serialize model to JSON
#model_json = model.model.to_json()
#with open(name + ".json", "w") as json_file:
#    json_file.write(model_json)
# serialize weights to HDF5
#model.model.save_weights(name + ".h5")
#print("\nSaved model to disk")

In [ ]:
#print(prediction)
#prediction = prediction_list[:][0][1]
#for i in range(forecast_horizon):
#    print(prediction[:][0][i])
a = np.empty(1)
a.fill(np.nan)
print(a)

Evaluation
==============================================================================================

In [ ]:
data = pd.DataFrame()
for i in range(forecast_horizon):
    a = np.empty(i)
    a.fill(np.nan)
    b = np.empty(forecast_horizon - i - 1)
    b.fill(np.nan)
    data['+'+str(i+1)+'h-prediction'] = np.append(np.append(a, prediction[:][0][i].values), b)#data[('prediction', 4.0)]

#data['prediction'] = pd.DataFrame(np.array(prediction).reshape([len(prediction), len(target_features)])).iloc[:,0]
data['measured'] = pd.DataFrame(np.array(testY).reshape([len(testY), len(target_features)])).iloc[:,0]
data = data.set_index(pd.MultiIndex.from_tuples(idx[:-2])).unstack()#[:-4]
data['pvlib'] = pvlib['2015-10-12 07:00:00':'2017-02-09 10:00:00'].reindex(data.index)

tmp = pd.DataFrame()
tmp[method] = data[method]
tmp['measured'] = data[('measured', 4.0)]
for i in range(forecast_horizon):
    tmp['+'+str(i+1)+'h-prediction'] = data['+'+str(i+1)+'h-prediction']
data = tmp
data.index = pd.to_datetime(data.index)
data = data.dropna()
#print(data)

m_col = data['measured']
l_col = data[method].dropna()

for horizon in range(1, forecast_horizon + 1):
    name = '+' + str(horizon) + 'h-prediction'
    p_col = data[name]

    walkForwardDailyLoss(m_col, p_col, l_col, method, name)
    scatter_predictions(m_col, p_col, name)

    print('%s test RMSE: %.3f' % (name, math.sqrt(mean_squared_error(m_col, p_col))))
    print('%s test RMSE: %.3f' % (method + ' forecast', math.sqrt(mean_squared_error(m_col, l_col))))
    draw_boxplot(m_col, p_col, l_col, method, name)
    draw_boxplot_monthly(m_col, p_col, l_col, method, name)

    m1, m2 = '2016-07-17 00:00:00', '2016-07-17 23:00:00'
    print('%s nice day RMSE: %.3f' % (name, math.sqrt(mean_squared_error(m_col[m1:m2], p_col[m1:m2]))))
    print('%s nice day RMSE: %.3f' % (method + ' forecast', math.sqrt(mean_squared_error(m_col[m1:m2], l_col[m1:m2]))))
    draw_boxplot(m_col, p_col, l_col, method, name, m1, m2)

    plot_timeseries(m_col, p_col, l_col, method, name, end='2015-10-19 07:00:00')
    plot_timeseries(m_col, p_col, l_col, method, name, start='2017-02-02 10:00:00')
    plot_timeseries(m_col, p_col, l_col, method, name, start=m1, end=m2)
    plot_timeseries(m_col, p_col, l_col, method, name)
    plot_timeseries(m_col, p_col, None, method, name)

    draw_histogram(p_col, m_col, name)

In [ ]:
draw_history(history)
draw_history(val_history, True)

In [ ]:
print(data.describe())
print(data.corr(method='pearson'))
print(data.corr(method='spearman'))
print(data.corr(method='kendall'))
data.to_csv(dir + 'predictions.csv', encoding='utf-8')

In [ ]:
if False:
    forcast_horizon = 3
    timesteps = 5
    testX = pd.DataFrame()
    testY = pd.DataFrame()
    testX['power t-5'] = [-4,-3,-2,-1,0,1,2,3,4,5]
    testX['power t-4'] = [-3,-2,-1,0,1,2,3,4,5,6]
    testX['power t-3'] = [-2,-1,0,1,2,3,4,5,6,7]
    testX['power t-2'] = [-1,0,1,2,3,4,5,6,7,8]
    testX['power t-1'] = [0,1,2,3,4,5,6,7,8,9]
    testX['test'] = [0,1,2,3,4,5,6,7,8,9]
    testY['power'] = [1,2,3,4,5,6,7,8,9,10]

    predictions = []
    for i in range(len(testX)):
        p = []
        for l in range(1, timesteps + 1):
            p.append(testX.iloc[i:i+1,:]['power t-'+str(l)].values[0])
        ps = []
        for f in range(forcast_horizon):
            t = testX.iloc[i+f:i+1+f,:].copy()
            for l in range(timesteps-1, 1, -1):
                t['power t-' + str(l+1)] = p[l]
                p[l] = p[l-1]
            p[0] = p[0] + 1.1
            ps.append(p[0])
        predictions.append(ps)
    print(predictions)